In [ ]:
# ================================================================
# INSTALL DEPENDENCIES
# ================================================================
!pip install prophet
!pip install lightgbm
!pip install pmdarima
!pip install matplotlib pandas numpy scikit-learn statsmodels tqdm

# ================================================================
# IMPORTS
# ================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
from statsmodels.tsa.statespace.sarimax import SARIMAX
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# ================================================================
# TASK 1: SIMULATED HOURLY DATA (3+ YEARS)
# ================================================================
np.random.seed(42)
date_range = pd.date_range(start="2020-01-01", end="2023-12-31 23:00", freq="H")
n = len(date_range)

# Trend
trend = 0.0005 * np.arange(n)

# Daily seasonality
daily = 10 * np.sin(2 * np.pi * (np.arange(n) % 24) / 24)

# Weekly seasonality
weekly = 7 * np.sin(2 * np.pi * (np.arange(n) % (24*7)) / (24*7))

# Noise
noise = np.random.normal(0, 2, n)

# Outliers
outliers = np.zeros(n)
outlier_idx = np.random.choice(n, size=int(n * 0.003), replace=False)
outliers[outlier_idx] = np.random.normal(30, 10, len(outlier_idx))

# Final series
y = 50 + trend + daily + weekly + noise + outliers

df = pd.DataFrame({
    "ds": date_range,
    "y": y
})

df.head()

# ================================================================
# TRAIN TEST SPLIT (Last 30 days = test)
# ================================================================
train = df.iloc[:-24*30]
test = df.iloc[-24*30:]

# ================================================================
# TASK 4: EXPANDING WINDOW TIME-SERIES CROSS VALIDATION
# ================================================================
def expanding_window_cv(data, n_folds=5, horizon=24):
    """
    Expanding window cross-validation.
    Each fold predicts 'horizon' hours ahead.
    """
    fold_size = len(data) // (n_folds + 1)
    metrics = []

    print("Running Expanding Window CV...")

    for i in tqdm(range(n_folds)):
        end_train = fold_size * (i + 1)
        cv_train = data.iloc[:end_train]
        cv_test = data.iloc[end_train:end_train + horizon]

        # ----------------------
        # Prophet
        # ----------------------
        p = Prophet(daily_seasonality=True, weekly_seasonality=True)
        p.fit(cv_train)
        future = p.make_future_dataframe(periods=horizon, freq="H")
        p_pred = p.predict(future).iloc[-horizon:]["yhat"].values

        # ----------------------
        # SARIMAX
        # ----------------------
        sarimax = SARIMAX(cv_train.y, order=(1,1,1), seasonal_order=(1,1,1,24))
        sarimax_fit = sarimax.fit(disp=False)
        s_pred = sarimax_fit.forecast(steps=horizon)

        # ----------------------
        # LightGBM
        # ----------------------
        lgb_train = cv_train.copy()
        lgb_train["hour"] = lgb_train["ds"].dt.hour
        lgb_train["dayofweek"] = lgb_train["ds"].dt.dayofweek

        lgb_test = cv_test.copy()
        lgb_test["hour"] = lgb_test["ds"].dt.hour
        lgb_test["dayofweek"] = lgb_test["ds"].dt.dayofweek

        model_lgb = LGBMRegressor()
        model_lgb.fit(lgb_train[["hour","dayofweek"]], lgb_train["y"])
        l_pred = model_lgb.predict(lgb_test[["hour","dayofweek"]])

        # ----------------------
        # METRICS
        # ----------------------
        actual = cv_test.y.values
        blended = (p_pred + s_pred + l_pred) / 3  # simple blend per fold

        mae = mean_absolute_error(actual, blended)
        rmse = np.sqrt(mean_squared_error(actual, blended))
        wape = np.sum(np.abs(actual - blended)) / np.sum(np.abs(actual))

        metrics.append([mae, rmse, wape])

    return pd.DataFrame(metrics, columns=["MAE", "RMSE", "WAPE"])

cv_results = expanding_window_cv(train)
print("\nCross Validation Results:")
cv_results, cv_results.mean()

# ================================================================
# TASK 2 & 3: TRAIN FINAL MODELS + STACKING ENSEMBLE
# ================================================================
# -------- Prophet --------
prophet = Prophet(daily_seasonality=True, weekly_seasonality=True)
prophet.fit(train)
future = prophet.make_future_dataframe(periods=24*30, freq="H")
p_forecast = prophet.predict(future).iloc[-24*30:]["yhat"].values

# -------- SARIMAX --------
sarimax_model = SARIMAX(train.y, order=(1,1,1), seasonal_order=(1,1,1,24))
sarimax_fit = sarimax_model.fit(disp=False)
s_forecast = sarimax_fit.forecast(steps=24*30)

# -------- LightGBM --------
train_lgb = train.copy()
train_lgb["hour"] = train_lgb["ds"].dt.hour
train_lgb["dow"] = train_lgb["ds"].dt.dayofweek

test_lgb = test.copy()
test_lgb["hour"] = test_lgb["ds"].dt.hour
test_lgb["dow"] = test_lgb["ds"].dt.dayofweek

lgb = LGBMRegressor()
lgb.fit(train_lgb[["hour","dow"]], train_lgb["y"])
l_forecast = lgb.predict(test_lgb[["hour","dow"]])

# -------- STACKING --------
stack_X_train = np.vstack([
    prophet.predict(train)[ "yhat" ].values,
    sarimax_fit.fittedvalues.values,
    lgb.predict(train_lgb[["hour","dow"]])
]).T

stack_y_train = train["y"].values

meta = Ridge()
meta.fit(stack_X_train, stack_y_train)

stack_X_test = np.vstack([p_forecast, s_forecast, l_forecast]).T
stack_pred = meta.predict(stack_X_test)

# ================================================================
# TASK 5: PREDICTION INTERVALS (BOOTSTRAP)
# ================================================================
boot_preds = []
for _ in range(50):
    noise = np.random.normal(0, 3, len(stack_pred))
    boot_preds.append(stack_pred + noise)

boot_preds = np.array(boot_preds)
lower = np.percentile(boot_preds, 5, axis=0)
upper = np.percentile(boot_preds, 95, axis=0)

# ================================================================
# FINAL OUTPUT TABLE
# ================================================================
final_output = pd.DataFrame({
    "ds": test["ds"].values,
    "Forecast": stack_pred,
    "Lower_90": lower,
    "Upper_90": upper
})

print(final_output.head())

# ================================================================
# PLOT FINAL FORECAST
# ================================================================
plt.figure(figsize=(15,5))
plt.plot(test["ds"], test["y"], label="Actual")
plt.plot(test["ds"], stack_pred, label="Forecast")
plt.fill_between(test["ds"], lower, upper, alpha=0.2)
plt.legend()
plt.title("Final 30-Day Forecast with 90% Prediction Intervals")
plt.show()
